In [1]:
import re, urllib, datetime, io
import pandas as pd
from scrapy import Spider, Request, signals
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from scrapy.http.cookies import CookieJar
from bs4 import BeautifulSoup as bs

In [2]:
class DataSpider(Spider): 
    name = "Data"
    
    # Settings 
    custom_settings = {'DOWNLOAD_DELAY': '1.5'}
    
    ts=datetime.datetime.now()
    
    # Regex patterns 
    patterns = {'people' : ".*/\d+\?sport_code\=MFB",
                'history' : ".*\/teams\/history\/MFB\/\d+",
                'roster' : ".*\/team\/\d+\/roster\/\d+",
                'stats' : ".*\/team\/\d+\/stats\/\d+",
               }
    
    # Files 
    _files = {'people':{'file':"People_{0}.csv".format(ts), 
                        'header':"team,person,link,\n",
                       }, 
              'history':{'file':"History_{0}.csv".format(ts),
                         'header':"team,link,\n",
                        }, 
              'roster':{'file':"Roster_{0}.csv".format(ts), 
                        'header':"team,link,\n",
                       }, 
              'stats':{'file':"Stats_{0}.csv".format(ts), 
                       'header':"team,link,\n"
                      },
             }
    
    newLinksFile = "NewLinks{0}.csv".format(ts)
    
    with open(newLinksFile, 'w') as f: 
        f.write('team,link,txt,key,match\n')
        
    # Table results 
    with open("Results.csv", "w") as f: 
        f.write("Team,Date,Opponent,Result\n")
                
    # Table Team stats 
    with open("TeamStats.csv", "w") as f: 
        f.write("Team,Stat,Rank,Value\n")
                
    # Individual 
    with open("Individual.csv", "w") as f: 
        f.write("Team,Stat,Player,Value\n")
                
    @classmethod
    def from_crawler(cls, crawler, *args, **kwargs):
        spider = super(DataSpider, cls).from_crawler(crawler, *args, **kwargs)
        crawler.signals.connect(spider.spider_closed, signal=signals.spider_closed)
        return spider
    
    def start_requests(self): 
        links = pd.read_csv("Teams_2017-12-02 14:55:51.037193.csv")
        links = links['link'].tolist()
        
        for link in links: 
            yield Request(url=link, 
                          callback=self.parse,
                          meta= {'REDIRECT_ENABLED': True}
                         )
    
    def parse(self, response): 
        # Team 
        team = response.selector.xpath("//body//div//fieldset//legend//a/text()").extract()[0]
        print(team)
        # Get all the links on the page
        le = LinkExtractor() 
        links = le.extract_links(response)
                
        # Extract the links and save to a list
        linksOut = []
        for link in links: 
            for k, pattern in self.patterns.items():
                if re.search(pattern, link.url) != None: 
                    linksOut.append({'team':team, 'link':link.url, 'txt':link.text, 'key':k, 'match':True})
        
        # Tables 
        tables = pd.read_html(response.body)
        
        # Results 
        tables[1].rename(columns=tables[1].iloc[1], inplace = True)
        tables[1].drop([0,1], inplace = True)
        tables[1]['team'] = team
                
        # Team stats 
        tables[2].rename(columns=tables[2].iloc[1], inplace = True)
        tables[2].drop([0,1], inplace = True)
        tables[2]['team'] = team
                
        # Individual stats
        tables[3].rename(columns=tables[3].iloc[1], inplace = True)
        tables[3].drop([0,1], inplace = True)
        tables[3]['team'] = team

        # Create a pandas df and append to csv 
        c = io.StringIO()
        df = pd.DataFrame(linksOut)
        df[['team', 'link', 'txt', 'key', 'match']].to_csv(c, index=False, header=False)
                
        io_results = io.StringIO()
        io_teamstats = io.StringIO()
        io_individual = io.StringIO()
                
        tables[1][['team', 'Date','Opponent','Result']].to_csv(io_results, index=False, header=False)
        tables[2][['team','Stat','Rank','Value']].to_csv(io_teamstats, index=False, header=False)
        tables[3][['team','Stat','Player','Value']].to_csv(io_individual, index=False, header=False)
        
        # Append  
        with open(self.newLinksFile, 'a') as f:
            f.write(c.getvalue())
        
        with open('Results.csv', 'a') as f: 
            f.write(io_results.getvalue())
            
        with open("TeamStats.csv", 'a') as f: 
            f.write(io_teamstats.getvalue())
        
        with open("Individual.csv", 'a') as f: 
            f.write(io_individual.getvalue())
        
    def spider_closed(self, spider): 
        print("Spider is done")
            

In [3]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
})

process.crawl(DataSpider)
process.start() 

2017-12-03 20:28:24 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-03 20:28:24 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}
2017-12-03 20:28:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-03 20:28:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddle

Air Force Falcons


2017-12-03 20:29:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/teams/62681> (referer: None)


Akron Zips


2017-12-03 20:29:03 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/teams/62682> (referer: None)


...

Western Mich. Broncos


2017-12-03 21:01:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/teams/83678> (referer: None)


Wisconsin Badgers


2017-12-03 21:01:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/teams/83679> (referer: None)
2017-12-03 21:01:14 [scrapy.core.engine] INFO: Closing spider (finished)
2017-12-03 21:01:14 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 578924,
 'downloader/request_count': 1082,
 'downloader/request_method_count/GET': 1082,
 'downloader/response_bytes': 3988380,
 'downloader/response_count': 1082,
 'downloader/response_status_count/200': 487,
 'downloader/response_status_count/302': 514,
 'downloader/response_status_count/500': 81,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 12, 4, 3, 1, 14, 235103),
 'httperror/response_ignored_count': 27,
 'httperror/response_ignored_status_count/500': 27,
 'log_count/DEBUG': 1134,
 'log_count/INFO': 66,
 'memusage/max': 146575360,
 'memusage/startup': 121135104,
 'response_received_count': 514,
 'retry/count': 54,
 'retry/max_reached': 27,
 'retry/reason_count/500 Inte

Wyoming Cowboys
Spider is done
